# Libraires

In [1]:
# import json file called config
import json
with open('config.json') as f:
    config = json.load(f)
TOKENIZER_ID = config['TOKENIZER_ID']
nrows = config['nrows']

In [2]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import PassiveAggressiveClassifier #used to train the model
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score, confusion_matrix, roc_auc_score, roc_curve # this is used to evaluate the model.
from sklearn.model_selection import train_test_split # to separate the dataset.
from sklearn.feature_extraction.text import TfidfVectorizer #used for the transformation of text data.
import pickle as pkl #file to save and load a model.
from sklearn.model_selection import GridSearchCV #pour l'optimisation des hyperparametres.
import time #pour étudier le temps d'execution d'entrainement, de notre meilleure modèle.
import psutil #pour calculer la mémoire.
import os #pour calculer le temps d'execution.
import joblib


# Load Data

In [3]:
data=pd.read_csv('title_text.csv',nrows=nrows).drop(['Unnamed: 0'],axis=1)

In [4]:
data

,title,text,isFake
0,U.S. immigration arrests up nearly 40 percent ...,NEW YORK (Reuters) - U.S. arrests of suspected...,False
1,Immigration judges exempt from Trump's federal...,(Reuters) - President Donald Trump’s federal h...,False
2,"In first remarks since retweet feud, UAE diplo...",DUBAI (Reuters) - A senior UAE diplomat said o...,False
3,"REVEALED: FBI Aided, Abetted ‘ISIS’ Terrorist ...",Daily Shooter 21st Century WireJust as ISIS is...,True
4,Bus bomb kills eight in Syria's Homs city: sta...,BEIRUT (Reuters) - A bomb blast killed eight p...,False
...,...,...,...
245,Trump’s Atty. Gen. Pick LIES About His Civil ...,Sen. Jeff Sessions (R-AL) is Trump s pick for ...,True
246,HERE’S WHY MEGA BAND U2 Is Blaming TRUMP For D...,,True
247,Second federal judge blocks Trump's curbs on t...,WASHINGTON (Reuters) - A second U.S. federal j...,False
248,"Rice chides Trump for criticism of judges, media",NEW YORK (Reuters) - Former U.S. Secretary of ...,False


In [5]:
#we make everything lower.
data['text']=data['text'].apply(lambda x: x.lower())
data

,title,text,isFake
0,U.S. immigration arrests up nearly 40 percent ...,new york (reuters) - u.s. arrests of suspected...,False
1,Immigration judges exempt from Trump's federal...,(reuters) - president donald trump’s federal h...,False
2,"In first remarks since retweet feud, UAE diplo...",dubai (reuters) - a senior uae diplomat said o...,False
3,"REVEALED: FBI Aided, Abetted ‘ISIS’ Terrorist ...",daily shooter 21st century wirejust as isis is...,True
4,Bus bomb kills eight in Syria's Homs city: sta...,beirut (reuters) - a bomb blast killed eight p...,False
...,...,...,...
245,Trump’s Atty. Gen. Pick LIES About His Civil ...,sen. jeff sessions (r-al) is trump s pick for ...,True
246,HERE’S WHY MEGA BAND U2 Is Blaming TRUMP For D...,,True
247,Second federal judge blocks Trump's curbs on t...,washington (reuters) - a second u.s. federal j...,False
248,"Rice chides Trump for criticism of judges, media",new york (reuters) - former u.s. secretary of ...,False


# Modification du text en données numériques.
Nous partons du principe que les données ont étés clean et que nous pouvons nous concerntrer à créer un model.

Comme les informations principales pour définir des Fake news et des vrai news, se base sur le texte et le titre, nous devons nous focaliser sur ces informations. Il faut ainsi que nous transformons le text et le titre en données afin que notre model puisse utiliser des données numériques pour trouver la bonne réponse. Nous allons utiliser la methode de bert-.
# Bert tokenizer

In [6]:
from transformers import BertTokenizer, BertModel


X = data['text']

# Create the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_bert = BertModel.from_pretrained('bert-base-uncased')

# Set the maximum sequence length
max_seq_length = 512

# Truncate or pad the tokenized sequences
X = X.apply(lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=max_seq_length, truncation=True, padding='max_length'))
print(X)

0      [101, 2047, 2259, 1006, 26665, 1007, 1011, 105...
1      [101, 1006, 26665, 1007, 1011, 2343, 6221, 839...
2      [101, 11558, 1006, 26665, 1007, 1011, 1037, 30...
3      [101, 3679, 13108, 7398, 2301, 7318, 29427, 20...
4      [101, 15335, 1006, 26665, 1007, 1011, 1037, 59...
                             ...                        
245    [101, 12411, 1012, 5076, 6521, 1006, 1054, 101...
246    [101, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...
247    [101, 2899, 1006, 26665, 1007, 1011, 1037, 211...
248    [101, 2047, 2259, 1006, 26665, 1007, 1011, 228...
249    [101, 4133, 2102, 8545, 1010, 12620, 1006, 266...
Name: text, Length: 250, dtype: object


In [23]:
y = data['isFake'].astype(int) #make it numerical
print(y.value_counts())

0    135
1    115
Name: isFake, dtype: int64


In [ ]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=512, truncation=True, padding="max_length") # Tokenize the text
    outputs = model_bert(**inputs) # Utiliser l'embedding du premier token ([CLS]) comme représentation du texte
    return outputs.last_hidden_state[:, 0, :].detach().numpy()

In [8]:
# we separate the trdata into training test and validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [9]:
X_train.shape

(640,)

In [10]:
X_test.shape,y_test.shape

((200,), (200,))

# Model Passive Agressive

Quand on parle d'un model passive aggressive, nous parlons d'un algorithme linéaire de classification et de regression. C'est un algorithme d'apprentissage en ligne, Il traite les données séquentiellement, ajustant le prédicteur à chaque nouvelle instance pour améliorer la prédiction future.

Le modèle se base sur deux principes:
- **Passive**: Si notre modèle prédit une instance juste, alors les poids et les biais ne seront pas changés. L'état actuelle de notre modèle sera maintenu.
- **Aggressive**: Si notre modèle fait une mauvaise prédiction sur une instance ressus, les poids et biais de notre modèle seront changés par le paramètre de régularisation **C**, le but étant de minimiser l'erreur. Il change les paramètres pour que la prédiction actuelle soit 'just'.

Le modèle utilise par défaut la fonction de cout **hinge**. Voici comment mathématiquement les poids et bias seront changés:

$$ w_{\text{new}} = w + \alpha y x $$
$$ b_{\text{new}} = b + \alpha y $$

où **w** est le vecteur de poids, **b** est le biais, **x** est le vecteur de caractéristiques de l'instance mal classée, **y** est l'étiquette correcte de l'instance, et **α** est le taux d'apprentissage calculé en fonction de l'erreur et du paramètre de régularisation **C**

Comme nous traitons des données de texte, et que nous devons faire une classification binaire, il ce va de dire que les text vont avoir des pattern differents, en utilisant le model passive aggressive, nous pouvons entrainer le model à définir une méthode pour les idenfifier.

In [25]:
# we build our model: C is the regulation parameter, and denotes the penalization the model will make on an incorrect prediciton.
model = PassiveAggressiveClassifier(C = 0.2, 
                                    random_state = 42,
                                    early_stopping=True,# we enable the early stopping procedure.
                                    n_iter_no_change=3, #number of iterations before early stopping
                                    verbose=True,
                                    loss='hinge', #fonction de cout
                                   )


In [26]:
filenm = 'PassiveAggressiveClassifier_model.pickle'

In [27]:
#on entraine le model sur les données d'entrainement.
model.fit(X_train,y_train)
#Create or open a file with write-binary mode and save the model to it
pickle = pkl.dump(model, open(filenm, 'wb'))

ValueError: setting an array element with a sequence.

In [ ]:
#on sort les predictions
#Step 2: Open the saved file with read-binary mode
model = pkl.load(open(filenm, 'rb'))
y_pred=model.predict(tfidf_test)

# Optimisation

Pour optimiser le model nous allons modifier les paramètres suivants:
- **C (Le paramètre de régularisation)**: Un des paramètres les plus importants de l'algorithme passive aggressive. Il est utilisé pour controler la marge d'erreur que notre modèle est près à accepté. Si on lui donne une valeur de plus en plus élevé il se peut que le modèle suraprenne, car il s'adapte mieux aux données d'entrainement. Ainsi une valeur plus faible serait favorable. Nous allons tester C avec les valeurs suivantes: **[0.01,0.05,0.1,0.5,1]**

In [ ]:
params_grid={'C':[0.01,0.05,0.1,0.5,1]}

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import make_scorer, accuracy_score


# Initialize the Passive Aggressive Classifier
model = PassiveAggressiveClassifier(random_state=42,early_stopping=True)

# On creer un scorer pour le grid search 
scorer = make_scorer(accuracy_score)

# Initialisation de la GridSearch pour trouver le meilleur C.
grid_search = GridSearchCV(estimator=model, param_grid=params_grid, scoring=scorer, cv=5)

# On lance les multiples entrainements.
grid_search.fit(tfidf_train, y_train)


In [ ]:
# Get the best combination of parameters
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_params}")
print(f"Best score: {best_score}")


# Model Evaluation

Afin d'évaluer le modèle nous faisons une représentation graphique des résultats obtenus. Nous allons étudier les metrics suivants:
- le **temps d'entrainement** que le modèle a besoin, ainsi que son **taux de mémoire (Mo)**.
- **matrice de confusion**, qui est un excellent choix d'évaluation de performance, car nous sommes dans le cas d'une classification binaire. 
- **L'accuracy** du model.
- **La précision**
- **recall**
- **F1-score**.
- **ROC-AUC score**

src: https://www.v7labs.com/blog/performance-metrics-in-machine-learning#h2

In [ ]:
#creation du modèle avec les meilleurs paramètres
best_model=PassiveAggressiveClassifier(C=best_params['C'],
                                       early_stopping=True,
                                       verbose=True,
                                       random_state=42)

Nous voullons étudier **le temps d'entrainement (en h/m/s)** et **l'utilisation de la mémoire (Mo)** que le modèle a besoin.

In [ ]:
def second_to_hms(seconds):
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return int(hours), int(minutes), int(seconds)

def bytes_to_Mo(mem_bytes):
    mem_kb = mem_bytes / 1024  # Convertir en kilooctets
    mem_mb = mem_kb / 1024  # Convertir en mégaoctets
    return mem_mb

In [ ]:
#lancemenet de l'enrestristrement de la mémoire.
process = psutil.Process(os.getpid())
mem_before_bytes = process.memory_info().rss

#lancement de l'enregistrement du temps d'entrainement.
start_time = time.time() 
# entrainement du modèle.
best_model.fit(tfidf_train,y_train)
end_time = time.time()

#le temps écoulé 
elapsed_time= end_time - start_time

print(f"Temps d'entrainement (h | m | s) : {second_to_hms(elapsed_time)[0]} | {second_to_hms(elapsed_time)[1]} | {second_to_hms(elapsed_time)[2]}")

mem_after_bytes = process.memory_info().rss
#on convertit les bytes en Mo.
mem_bytes=mem_after_bytes - mem_before_bytes


print(f"Utilisation de la mémoire (Mo) : {bytes_to_Mo(mem_bytes)}")

In [ ]:
#Create or open a file with write-binary mode and save the model to it
pickle = pkl.dump(best_model, open(filenm, 'wb'))

In [ ]:
#optention des prédictions de ce modèle.
y_pred=best_model.predict(tfidf_test)

Nous déterminons la **matrice de confusion**.

In [ ]:
#creation de la matrice de confusion
cm = confusion_matrix(y_test, y_pred)

#représentation graphique du résultat du meilleure model
sns.heatmap(cm, annot=True)
plt.xlabel('Les labels prédits')
plt.ylabel('Les vrais labels')
plt.title('Matrice de confusion')
plt.show()

In [ ]:
print(f"le modèle a donc {cm[0][0]} instances vrai Positives et {cm[1][1]} instances de Vrai négatives, {cm[1][0]} instances de Faux Positives et {cm[0][1]} instances de Faux négatives. ")

Nous allons avant tout calculer **la précision globale (accuracy)** de notre modèle. 

In [ ]:
acc=accuracy_score(y_pred,y_test)
print(f"Notre modèle a une précision globale de {round(acc*100,2)}%, ce qui signifie qu'il prédit correctement les classes des instances dans {round(acc*100,2)}% des cas.")

on calcul le **precision**, c'est la capacité du modèle à correctement identifier les instances que nous avons prédits. Donc de prédire qu'une News vrai soit vraie.

In [ ]:
precision=precision_score(y_test,y_pred)
print(f"notre model a une precision de {round(precision*100,2)}% . lorsqu'il prédit une classe comme positive, il a raison dans {round(precision*100,2)}% des cas.")

on calcul le **recall**, ce qui es le pourcentage de Vrai positif **(dans notre cas les Vrai news)** que notre modèle arrive à prédire.

In [ ]:
recall=recall_score(y_test,y_pred)
print(f"notre model arrive a détecter {round(recall*100,2)}%. Donc sur l'ensemble des vrai positives, le modèle parvient à en identifier correctement {round(recall*100,2)}% des cas.")

On calcule le **F1-score**, c'est une combinaison de la précision et du recall, en calculant la moyenne harmonique entre les deux. Son but est **d'étudier l'équilibre entre la precision et le recall.**

In [ ]:
f1score=f1_score(y_test,y_pred)
print(f"notre model a un F1-score de {round(f1score*100,2)}%, ce qui signifie que il y a un excellent equilibre entre le recall et la précision.")

## Courbe ROC-AUC
Dans le contexte d'une classification binaire, il est essentiel d'utiliser la courbe ROC-AUC pour évaluer notre modèle. Nous allons examiner deux aspects :

- **ROC (Receiver Operating Characteristic)**: Cette courbe illustre le rapport entre le taux de vrais positifs et le taux de faux positifs. Elle permet de visualiser la capacité du modèle à discriminer entre les deux classes.

- **AUC (Area Under the Curve)**: C'est une mesure qui résume la courbe ROC en calculant l'aire sous celle-ci et au-dessus de la diagonale qui va du coin inférieur gauche au coin supérieur droit du graphique. Un score AUC proche de 1 indique que notre modèle fait des prédictions presque parfaites. En revanche, un score proche de 0,5 suggère que nos prédictions sont principalement due au hasard, ce qui signifie que le modèle n'apprend pas de manière efficace.

L'objectif de ces mesures est de déterminer à quel point notre modèle est efficace dans la tâche de classification binaire.

On définit le score ROC-AUC.

In [ ]:
roc_auc=roc_auc_score(y_test,y_pred)
roc_auc

In [ ]:
print(f"Le modèle a donc une score ROC-AUC de {roc_auc} ce qui est très proche de 1, donc le modèle a une excellente capacité à différencier entres les Fake et les vrais news.")

Avec une valeur aussi élevée, nous pouvons être confiants dans le fait que notre modèle est extrêmement précis et fiable pour notre tâche de classification, avec très peu de faux positifs et de faux négatifs. Cela indique que notre modèle fait des erreurs de prédiction minimales. Nous allons maintenant procéder à la représentation graphique de la courbe ROC-AUC.

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Taux de Faux Positives (%)')
plt.ylabel('Taux de Vrai Positives (%)')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

L'axe des abscisses (axe x) représente le taux de faux positifs, c'est-à-dire le pourcentage de vraies nouvelles que notre modèle a incorrectement classifiées comme fausses (Fake News).

L'axe des ordonnées (axe y) indique le taux de vrais positifs, qui correspond au pourcentage de fausses nouvelles que notre modèle a correctement identifiées comme telles.

La courbe montante vers le coin supérieur gauche du graphique suggère que le modèle parvient à maintenir un taux élevé de vrais positifs tout en minimisant le taux de faux positifs, ce qui est le comportement idéal pour un classificateur. 
Cette représentation représente la robustesse du mdoèle.

# Test de performances sur des données générées par Téléchargé ailleurs

In [ ]:
filenm = 'PassiveAggressiveClassifier_model.pickle'
tfidf_vectorizer = joblib.load('tfidf_vectorizer.pkl')
best_model=pkl.load(open(filenm, 'rb'))

In [ ]:
validation_data=pd.read_csv("data/validation_data.csv")
validation_data.head()

In [ ]:
validation_data=validation_data[['text','isFake']]
validation_data

In [ ]:
# Prepare the data
validation_data_X = validation_data['text'].apply(lambda x: x.lower())  # the text data
validation_data_y=validation_data['isFake']

# Transform the data using the TF-IDF Vectorizer
X_tfidf = tfidf_vectorizer.transform(validation_data_X)

# Use the trained model to make predictions on the dataset
predictions = best_model.predict(X_tfidf)

In [ ]:
confusion_matrix(validation_data_y,predictions)

In [ ]:
accuracy_score(validation_data_y,predictions)

# Conclusion

les évaluations et analyses approfondies de notre modèle confirment qu'il est extrêmement efficace et fiable pour l'identification des fake news. Ses performances élevées sur plusieurs métriques clés (accuracy,precision,recall,F1-score et ROC-AUC) démontrent sa capacité à fournir des prédictions précises et équilibrées, confortant notre choix d'utiliser ce modèle pour cette tache.